# 1. Imports & Environment Setup

- **Core Imports:**
  - `os`, `warnings`, `datetime`, `uuid`, `string`, `random`, `hashlib`, `traceback`.

- **LangChain/LangGraph Imports:**
  - `ChatOpenAI`, `StreamingStdOutCallbackHandler`, `TavilySearch`, `TavilySearchResults`, agent/graph/integration tools, message types.

- **Notebook & Display:**
  - `clear_output`, `Markdown` from `IPython.display`.

- **Environment:**
  - Suppresses TensorFlow warnings.
  - Loads environment variables (e.g., API keys) using `python-dotenv`.


In [1]:
import os
import warnings
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_tavily import TavilySearch
from langchain_community.tools import TavilySearchResults
from langchain.agents import create_tool_calling_agent, AgentExecutor
from typing import TypedDict, Dict, List
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from datetime import datetime
from IPython.display import clear_output, Markdown
import traceback
import uuid
import string
import random
import hashlib

# 2. Key Utility Functions

- **`get_api_key(key_name="OPENROUTER_API_KEY")`**  
  Retrieves an API key from environment variables. Raises an error if the key is not found.

- **`initialize_llm(...)`**  
  Initializes the language model (LLM) with streaming and callback support.

- **`save_file(data, filename, uniqueness_method="uuid")`**  
  Saves output (e.g., generated plans) to a Markdown file using one of several unique-naming strategies (`uuid`, `random_string`, `hash`, `counter`, or `datetime`).

- **`show_md_file(file_path)`**  
  Displays a Markdown file’s content within the notebook using IPython's `Markdown` display function.


In [2]:
# Suppress TensorFlow and CUDA warnings for a cleaner notebook output
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
warnings.filterwarnings("ignore")

# Load all environment variables (such as API keys) from .env file
load_dotenv()

def get_api_key(key_name="OPENROUTER_API_KEY"):
    """
    Retrieve an API key from environment variables.

    Args:
        key_name (str): Name of the environment variable.

    Returns:
        str: Retrieved API key.

    Raises:
        ValueError: If no key is found.
    """
    api_key = os.getenv(key_name)
    if not api_key:
        raise ValueError(f"Invalid API key: {key_name} not found in environment variables")
    return api_key

def initialize_llm(model_name="meta-llama/llama-3.1-8b-instruct", temperature=0.4, use_streaming=True):
    """
    Initialize a streaming Large Language Model (LLM) with specified parameters.

    Args:
        model_name (str): The model's name.
        temperature (float): Sampling temperature for generation.
        use_streaming (bool): Whether to enable streaming output.

    Returns:
        ChatOpenAI: Instance of the language model.
    """
    api_key = get_api_key()
    callbacks = [StreamingStdOutCallbackHandler()]
    llm = ChatOpenAI(
        model_name=model_name,
        temperature=temperature,
        streaming=use_streaming,
        callbacks=callbacks,
        openai_api_key=api_key,
        openai_api_base="https://openrouter.ai/api/v1"
    )
    return llm

# Initialize the language model for use throughout the notebook
llm = initialize_llm()

python-dotenv could not parse statement starting at line 12


In [3]:
def save_file(data, filename, uniqueness_method="uuid"):
    """
    Save data as a markdown file in a uniquely named manner.

    Args:
        data (str): Content to save.
        filename (str): Base filename.
        uniqueness_method (str): How to make filenames unique (uuid, random_string, hash, counter, datetime).

    Returns:
        str: Path to the saved file.
    """
    folder_name = "Travel Plans"
    os.makedirs(folder_name, exist_ok=True)  # Ensure folder exists

    # Choose unique identifier method for filename
    if uniqueness_method == "uuid":
        unique_id = str(uuid.uuid4())[:8]
    elif uniqueness_method == "random_string":
        chars = string.ascii_letters + string.digits
        unique_id = ''.join(random.choice(chars) for _ in range(8))
    elif uniqueness_method == "hash":
        content_hash = hashlib.md5((data + str(datetime.now())).encode()).hexdigest()[:8]
        unique_id = content_hash
    elif uniqueness_method == "counter":
        counter = 1
        while True:
            test_filename = f"{filename}_{counter}.md"
            test_path = os.path.join(folder_name, test_filename)
            if not os.path.exists(test_path):
                unique_id = str(counter)
                break
            counter += 1
    elif uniqueness_method == "datetime":
        unique_id = datetime.now().strftime("%Y%m%d%H%M%S")
    else:
        unique_id = str(uuid.uuid4())[:8]

    final_filename = f"{filename}_{unique_id}.md"
    file_path = os.path.join(folder_name, final_filename)

    # Write data to file
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(data)
        print(f"File '{file_path}' created successfully.")

    return file_path

def show_md_file(file_path):
    """
    Display the content of a markdown file within the notebook.

    Args:
        file_path (str): Path to the markdown file.
    """
    with open(file_path, "r", encoding = "utf-8") as file:
        content = file.read()
    display(Markdown(content))

# 3. Planner State Structure
Defines a TypedDict, PlannerState, which holds all state for the travel planning workflow:

- travel_preferences: User’s trip preferences.
- destination_options: Suggested destinations.
- selected_destination: User’s chosen destination.
- budget_plan: Trip budget breakdown.
- itinerary: Day-by-day trip plan.
- bookings: Booking recommendations.
- feedback: User feedback.
- final_trip: The final trip package.
messages: Message history (for context).
status: Workflow status string.

In [4]:
class PlannerState(TypedDict):
    """
    Structure representing the entire travel planning state across all workflow nodes.
    """
    travel_preferences: Dict
    destination_options: List[Dict]
    selected_destination: Dict
    budget_plan: Dict
    itinerary: List[Dict]
    bookings: Dict
    feedback: Dict
    final_trip: Dict
    messages: List
    status: str

# 4. Workflow Node Functions
Each node is a function that takes and returns a PlannerState object, incrementally enriching the plan:

- input_collector:
    - Extracts user travel preferences from the message history via the LLM.
- destination_research: 
    - Suggests 3–5 destinations matching the user’s preferences, with descriptions, costs, and timing.
- collect_selected_destination:
    - Captures which destination the user selects from the options.
- budget_planner:
    - Breaks down the trip’s expected costs (transport, accommodation, activities, etc.).
- itinerary_builder:
    - Generates a detailed multi-day plan, balancing activity and rest.
- booking_assistant: 
    - Provides booking recommendations (flights, hotels, activities, insurance, etc.), with tips and links.
- trip_exporter: 
    - Compiles the entire trip—including preferences, itinerary, budget, bookings, packing tips, and emergency contacts—into a final Markdown document.



In [5]:
def input_collector(state: PlannerState) -> PlannerState:
    """
    Collects and organizes user's travel preferences from messages.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state with extracted preferences.
    """
    messages = state["messages"]
    # Extract the latest user message
    human_message = next((m for m in reversed(messages) if isinstance(m, HumanMessage)), None)
    system_prompt = (
        "You are a travel input collector. Extract travel preferences from the user's message.\n"
        "Include: destination interests, travel dates, budget range, number of travelers, "
        "accommodation preferences, activity interests, and any special requirements."
    )
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        human_message
    ])
    preferences = {
        "raw_input": human_message.content,
        "processed_preferences": response.content,
        "timestamp": datetime.now().isoformat()
    }
    
    #Update state
    state["travel_preferences"] = preferences
    state["messages"].append(AIMessage(content="I've collected your travel preferences. Moving on to destination research."))
    state["status"] = "preferences_collected"
    return state

In [6]:
def destination_research(state: PlannerState) -> PlannerState:
    """
    Research and suggest possible destinations based on user preferences.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state with destination suggestions.
    """
    preferences = state["travel_preferences"]
    system_prompt = (
        "You are a destination research expert. Based on the user's travel preferences, suggest "
        "3-5 suitable destinations. For each destination, provide:\n"
        "- Name and brief description\n"
        "- Why it matches their preferences\n"
        "- Best time to visit\n"
        "- Estimated overall cost level\n"
        "- Key attractions"
    )
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Travel Preferences: {preferences}")
    ])
    destination_options = {
        "options": response.content,
        "research_timestamp": datetime.now().isoformat()
    }
    state["destination_options"] = destination_options
    state["messages"].append(AIMessage(content="I've researched some destinations that match your preferences. Moving on to budget planning."))
    state["status"] = "awaiting_destination_selection"
    return state

In [7]:
def collect_selected_destination(state: PlannerState) -> PlannerState:
    """
    Capture the user's selected destination from the message history.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state with the user's destination choice.
    """
    messages = state["messages"]
    latest_human = next((m for m in reversed(messages) if isinstance(m, HumanMessage)), None)
    if not latest_human:
        raise ValueError("No user input found for destination selection.")

    selected_destination = {
        "name": latest_human.content,
        "details": "To be looked up or parsed from previous suggestions."
    }
    state["selected_destination"] = selected_destination
    state["messages"].append(AIMessage(content=f"Thanks! You've selected **{latest_human.content}**. We'll proceed to budget planning."))
    state["status"] = "destination_selected"
    return state

In [8]:
def budget_planner(state: PlannerState) -> PlannerState:
    """
    Create a detailed budget breakdown for the user's trip.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state with the trip budget plan.
    """
    preferences = state["travel_preferences"]
    destination = state["selected_destination"]
    system_prompt = (
        "You are a travel budget planner. Create a detailed budget breakdown for the trip, including:\n"
        "- Transportation (flights, local transit)\n"
        "- Accommodation costs\n"
        "- Daily expenses (food, activities)\n"
        "- Recommended spending money\n"
        "- Potential extras and contingencies"
    )
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Travel preferences: {preferences}, Selected destination: {destination}")
    ])
    budget_plan = {
        "breakdown": response.content,
        "timestamp": datetime.now().isoformat()
    }
    state["budget_plan"] = budget_plan
    state["messages"].append(AIMessage(content="I've created a budget plan for your trip. Now building your itinerary."))
    state["status"] = "budget_planned"
    return state

In [9]:
def itinerary_builder(state: PlannerState) -> PlannerState:
    """
    Build a detailed day-by-day itinerary for the trip.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state with the trip itinerary.
    """
    preferences = state["travel_preferences"]
    destination = state["selected_destination"]
    budget = state["budget_plan"]
    system_prompt = (
        "You are a travel itinerary expert. Create a detailed day-by-day itinerary for the trip, including:\n"
        "- Arrival and departure logistics\n"
        "- Daily activities and attractions\n"
        "- Meal recommendations\n"
        "- Rest periods and flexibility\n"
        "- Special experiences or highlights\n"
        "Balance the itinerary to avoid exhaustion while maximizing experiences within budget constraints."
    )
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Travel preferences: {preferences}, Selected destination: {destination}, Budget plan: {budget}")
    ])
    itinerary = {
        "daily_plan": response.content,
        "timestamp": datetime.now().isoformat()
    }
    state["itinerary"] = itinerary
    state["messages"].append(AIMessage(content="Your itinerary is ready! Moving on to booking assistance."))
    state["status"] = "itinerary_built"
    return state

In [10]:
def booking_assistant(state: PlannerState) -> PlannerState:
    """
    Provide booking recommendations and strategies for the trip.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state with booking information.
    """
    preferences = state["travel_preferences"]
    destination = state["selected_destination"]
    itinerary = state["itinerary"]
    system_prompt = (
        "You are a travel booking assistant. Provide detailed booking recommendations, including:\n"
        "- Flight options and booking strategies\n"
        "- Accommodation recommendations with booking links/platforms\n"
        "- Activity reservation requirements\n"
        "- Transportation booking needs\n"
        "- Travel insurance options\n"
        "Include tips for getting the best deals and necessary booking timelines."
    )
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Travel preferences: {preferences}, Selected destination: {destination}, Itinerary: {itinerary}")
    ])
    bookings = {
        "recommendations": response.content,
        "timestamp": datetime.now().isoformat()
    }
    state["bookings"] = bookings
    state["messages"].append(AIMessage(content="Here are your booking recommendations. Please provide feedback on the plan."))
    state["status"] = "bookings_provided"
    return state

In [11]:
def trip_exporter(state: PlannerState) -> PlannerState:
    """
    Compile all trip details into a final exportable document.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state containing the final trip package.
    """
    preferences = state["travel_preferences"]
    destination = state["selected_destination"]
    budget = state["budget_plan"]
    itinerary = state["itinerary"]
    bookings = state["bookings"]
    system_prompt = (
        "You are a travel document preparation specialist. Create a comprehensive trip package including:\n"
        "- Trip summary with key details\n"
        "- Complete itinerary with timings\n"
        "- Budget breakdown\n"
        "- Booking details and confirmation needs\n"
        "- Packing recommendations\n"
        "- Travel tips specific to the destination\n"
        "- Emergency contacts and resources\n"
        "Format this as a complete travel document that the traveler can use throughout their journey."
    )
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"""
        Travel preferences: {preferences}
        Selected destination: {destination}
        Budget plan: {budget}
        Itinerary: {itinerary}
        Booking details: {bookings}
        """)
    ])
    final_trip = {
        "package": response.content,
        "created_at": datetime.now().isoformat()
    }
    state["final_trip"] = final_trip
    state["messages"].append(AIMessage(content="Your trip package is ready! Here's everything you need for your journey."))
    state["status"] = "trip_exported"
    return state

# 5. Workflow Construction with LangGraph

- Adds each function as a node to a StateGraph.
- Specifies node execution order (edges).
- Entry point: input_collector.
- Terminal node: trip_exporter (signals workflow completion).
- Compiles the workflow into a graph object.

In [12]:
from langgraph.graph import StateGraph, END

# Create the workflow graph structure
workflow = StateGraph(PlannerState)

# Add all workflow nodes
workflow.add_node("input_collector", input_collector)
workflow.add_node("destination_research", destination_research)
workflow.add_node("collect_selected_destination", collect_selected_destination)
workflow.add_node("budget_planner", budget_planner)
workflow.add_node("itinerary_builder", itinerary_builder)
workflow.add_node("booking_assistant", booking_assistant)
workflow.add_node("trip_exporter", trip_exporter)

# Define workflow logic (edges)
workflow.set_entry_point("input_collector")
workflow.add_edge("input_collector", "destination_research")
workflow.add_edge("destination_research", "collect_selected_destination")
workflow.add_edge("collect_selected_destination", "budget_planner")
workflow.add_edge("budget_planner", "itinerary_builder")
workflow.add_edge("itinerary_builder", "booking_assistant")
workflow.add_edge("booking_assistant", "trip_exporter")
workflow.add_edge("trip_exporter", END)

# Compile workflow for execution
graph = workflow.compile()

# 6. Main Interaction Loop
- process_travel_query(query, planner_state):
    - Handles one user query, advancing the workflow, updating state, and returning assistant responses (or errors).
    - Accepts "exit" to terminate.
    - "continue" advances to next step.
- Sample Interaction:
    - Simulates a conversation, starting with a user’s preferences, continuing through destination selection, and ending with trip export and Markdown file creation.

In [13]:
def process_travel_query(query, planner_state):
    """
    Process a single user query, advancing the workflow and updating state.

    Args:
        query (str): The user's input query.
        planner_state (PlannerState): The current state of the planner.

    Returns:
        tuple: (response (str), updated_state (PlannerState))
    """
    if query.lower() == "exit":
        return "Exiting the travel planner.", planner_state

    # Add new user input to the state, unless just continuing
    if query.lower() != "continue":
        planner_state["messages"].append(HumanMessage(content=query))

    # Debug output for status and current messages
    print(f"\n[Debug] Current status: {planner_state['status']}")
    print(f"[Debug] Current messages: {[msg.content for msg in planner_state['messages']]}")

    try:
        assistant_response = ""
        for output in graph.stream(planner_state):
            for node, updated_state in output.items():
                print(f"\n[Node: {node}]")
                # Collect new assistant messages
                for message in updated_state["messages"]:
                    if isinstance(message, AIMessage) and message not in planner_state["messages"]:
                        assistant_response += f"{message.content}\n"
                planner_state = updated_state

                print(f"[Debug] Updated status after {node}: {planner_state['status']}")

                # If workflow is complete, export the trip package
                if planner_state["status"] == "trip_exported":
                    final_package = planner_state["final_trip"].get("package", "No package available.")
                    assistant_response += f"\nFinal Trip Package:\n{final_package}\n\nPlanning complete!"
                    return assistant_response, planner_state
        # Prompt user based on workflow state
        if not assistant_response:
            if planner_state["status"] == "initial":
                assistant_response = "Enter your travel preferences (e.g., destination, dates, budget, etc.): "
            elif planner_state["status"] == "awaiting_destination_selection":
                options = planner_state.get("destination_options", {}).get("options", "No options available yet.")
                assistant_response = f"\nSuggested destinations:\n{options}\nPlease select a destination from the suggested options: "
            else:
                assistant_response = "Provide any additional input or feedback (or type 'continue' to proceed): "

        return assistant_response, planner_state

    except Exception as e:
        error_message = (
            f"Error occurred: {e}\nStack trace:\n{traceback.format_exc()}\n"
            "Please check your setup (e.g., API keys, dependencies) and try again."
        )
        return error_message, planner_state

In [14]:
# Initial empty state for the workflow
state = PlannerState(
    travel_preferences={},
    destination_options=[],
    selected_destination={},
    budget_plan={},
    itinerary=[],
    bookings={},
    feedback={},
    final_trip={},
    messages=[],
    status="initial"
)

#simulate several queries
queries = [
    "I want to travel to Kenya in June 2025 with a budget of $2000. It should be a family trip. We love water activities and safaris",
    "continue",
    "Nairobi sounds good, let's select it.",
    "exit"
]

print("Welcome to the Travel Planning Assistant!")
current_state = state  # Use the initialized state

for query in queries:
    response, current_state = process_travel_query(query, current_state)
    print(f"\nAssistant: {response}")

    # Save and show response if workflow is complete
    if query.lower() == "exit" or current_state["status"] == "trip_exported":
        file_path = save_file(response, filename="travel_plan", uniqueness_method="datetime")
        clear_output()
        show_md_file(file_path)
        break


Final Trip Package:
**Kenya Family Trip Document**

**Trip Summary**

* Destination: Kenya
* Travel Dates: June 2025
* Budget: $2000
* Number of Travelers: Family (3-5 people)
* Accommodation Preferences: Family-friendly hotel or resort
* Activity Interests: Water activities (snorkeling, diving, swimming) and safaris (game drives, birdwatching)
* Special Requirements: None specified

**Itinerary**

**Day 1: Arrival in Nairobi**

* Arrive at Jomo Kenyatta International Airport (NBO) in Nairobi, Kenya
* Transfer to a family-friendly hotel in the city center (e.g. Hotel Boulevard or similar)
* Spend the day relaxing and acclimating to the local time zone
* Dinner: Try local cuisine at a nearby restaurant, such as Nyama Choma or similar (budget $20-$30 per person)

**Day 2: Nairobi National Park and Giraffe Centre**

* Morning: Visit Nairobi National Park for a game drive (approx. $50-$100 per person)
* Afternoon: Visit the Giraffe Centre for a unique wildlife experience (approx. $20-$30 per person)
* Return to the hotel for dinner and rest

**Day 3: Lake Naivasha and Water Activities**

* Morning: Drive to Lake Naivasha (approx. 2 hours) and check-in at a family-friendly resort (e.g. Lake Naivasha Resort or similar)
* Afternoon: Enjoy water activities such as snorkeling, kayaking, or paddleboarding (approx. $50-$100 per person)
* Dinner: Enjoy a seafood dinner at a local restaurant (budget $30-$50 per person)

**Day 4: Hell's Gate National Park and Cycling**

* Morning: Visit Hell's Gate National Park for a cycling tour (approx. $50-$100 per person)
* Afternoon: Explore the park on foot or by bike and enjoy the scenic views
* Return to the resort for dinner and rest

**Day 5: Maasai Mara National Reserve**

* Morning: Drive to Maasai Mara National Reserve (approx. 5 hours) and check-in at a family-friendly safari lodge (e.g. Mara Serena Safari Lodge or similar)
* Afternoon: Enjoy a game drive in the reserve (approx. $100-$200 per person)
* Dinner: Enjoy a bush dinner at the lodge (budget $50-$100 per person)

**Day 6: Maasai Mara National Reserve**

* Morning: Enjoy a hot air balloon ride over the reserve (approx. $200-$300 per person)
* Afternoon: Continue with a game drive and explore the reserve's diverse wildlife
* Return to the lodge for dinner and rest

**Day 7: Departure**

* Morning: Depart for Nairobi (approx. 5 hours) and transfer to the airport for departure

**Budget Breakdown**

* Transportation: $1,600 - $2,700
* Accommodation: $1,400 - $2,800
* Daily Expenses: $900 - $2,750
* Total Budget: $4,300 - $8,250

**Booking Details**

* Flights: Book flights from your preferred departure city to Nairobi, Kenya (NBO) in June 2025. Consider flying into Nairobi on a Tuesday or Wednesday for better rates.
* Accommodations: Book the Hotel Boulevard or a similar family-friendly hotel in the city center for Nairobi, Lake Naivasha Resort or a similar family-friendly resort for Lake Naivasha, and Mara Serena Safari Lodge or a similar family-friendly safari lodge for Maasai Mara National Reserve.
* Activities and Tours: Book the Nairobi National Park game drive and Giraffe Centre visit through a reputable tour operator like African Safaris or Kenya Wildlife Service. Book the hot air balloon ride over Maasai Mara National Reserve through a reputable operator like Balloons Over the Mara.
* Transportation: Book a private transfer service from the airport to your hotel in Nairobi. Consider booking a private car and driver through a reputable operator like African Safaris or Kenya Wildlife Service for transportation between destinations.
* Travel Insurance: Consider purchasing travel insurance that covers medical, trip cancellations, and delays.

**Packing Recommendations**

* Pack accordingly for the season and activities planned
* Consider packing:
	+ Lightweight and breathable clothing
	+ Comfortable shoes for walking and hiking
	+ Sun protection (sunscreen, hats, sunglasses)
	+ Insect repellent
	+ Power adapter for charging electronic devices
	+ Camera and charger for capturing memories

**Travel Tips**

* Research and book activities and tours in advance to ensure availability and get the best prices
* Consider budgeting extra for any additional activities or expenses that may arise during the trip
* Pack accordingly for the season and activities planned
* Be mindful of local customs and traditions
* Stay hydrated and bring sunscreen to protect yourself from the sun

**Emergency Contacts and Resources**

* Emergency contact: [insert contact information for your emergency contact]
* Travel insurance provider: [insert contact information for your travel insurance provider]
* Local authorities: [insert contact information for local authorities in case of an emergency]
* Medical facilities: [insert contact information for local medical facilities in case of a medical emergency]

**Additional Resources**

* Kenya Tourism Board: [insert website or contact information for the Kenya Tourism Board]
* African Safaris: [insert website or contact information for African Safaris]
* Kenya Wildlife Service: [insert website or contact information for the Kenya Wildlife Service]

Note: The above estimates are based on a family of 3-5 people and assume a moderate level of spending. The actual costs may vary depending on individual preferences and exchange rates.

Planning complete!

In [15]:
print(langchain.__version__)

NameError: name 'langchain' is not defined

In [16]:
print(langgraph.__version__)

NameError: name 'langgraph' is not defined